<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#(a)" data-toc-modified-id="(a)-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>(a)</a></span></li><li><span><a href="#(b)" data-toc-modified-id="(b)-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span>(b)</a></span></li><li><span><a href="#(c)" data-toc-modified-id="(c)-0.3"><span class="toc-item-num">0.3&nbsp;&nbsp;</span>(c)</a></span></li><li><span><a href="#(d)" data-toc-modified-id="(d)-0.4"><span class="toc-item-num">0.4&nbsp;&nbsp;</span>(d)</a></span></li></ul></li></ul></div>

Use the method of Steepest Descent with $TOL = 0.05$ to approximate the solutions of the following nonlinear systems.

The nonlinear system continuous with ex10.3.3

In [1]:
import numpy as np
from numpy import linalg
from abc import abstractmethod
import pandas as pd
import math

pd.options.display.float_format = '{:,.8f}'.format
np.set_printoptions(suppress=True, precision=8)

TOR = pow(10.0, -6)
MAX_ITR = 150

In [2]:
class SteepestDescentMethod(object):

    def __init__(self):
        return

    @abstractmethod
    def f(self, x):
        return NotImplementedError('Implement f()!')

    @abstractmethod
    def g(self, x):
        return NotImplementedError('Implement g()!')

    @abstractmethod
    def grad_g(self, x):
        return NotImplementedError('Implement grad_g()!')

    @abstractmethod
    def jacobian(self, x):
        return NotImplementedError('Implement jacobian()!')

    @abstractmethod
    def run(self, x):
        return NotImplementedError('Implement run()!')

## (a) 
$$\begin{align*}
4x_1^2 - 20 x_1 + \frac{1}{4} x_2^2 + 8 &= 0 \\
\frac{1}{2}x_1x_2^2+2x_1-5x_2+8 &= 0
\end{align*}$$

In [3]:
class SteepestDescent(SteepestDescentMethod):

    def __init__(self):
        super(SteepestDescentMethod, self).__init__()

    def f(self, x):
        sol = np.zeros(len(x))
        sol[0] = 4 * pow(x[0], 2) - 20 * x[0] + pow(x[1], 2) / 4 + 8
        sol[1] = x[0] * pow(x[1], 2) / 2 + 2 * x[0] - 5 * x[1] + 8
        return sol

    def jacobian(self, x):
        jac = np.zeros(shape=(2, 2))
        jac[0][0] = 8 * x[0] - 20
        jac[0][1] = x[1] / 2
        jac[1][0] = pow(x[1], 2) / 2
        jac[1][1] = x[0] * x[1] - 5
        return jac

    def g(self, x):
        sol = self.f(x)
        return sum([e * e for e in sol])

    def grad_g(self, x):
        return 2 * self.jacobian(x).transpose().dot(self.f(x))

    def run(self, x):
        df = pd.DataFrame(columns=['x' + str(i + 1) for i in range(len(x))] + ['g', 'residual', 'actual-residual'])

        row = len(df)
        df.loc[row] = [xe for xe in x] + [self.g(x), np.nan, np.nan]

        while True:
            prev_x = x
            g1 = self.g(x)
            z = self.grad_g(x)
            z0 = linalg.norm(z, 2)
            if z0 == 0.0:
                print('Zero gradient')
                return x

            z /= z0
            alpha3 = 1
            g3 = self.g(x - alpha3 * z)
            while g3 >= g1:
                alpha3 /= 2.0
                g3 = self.g(x - alpha3 * z)
                if alpha3 < TOR / 2.0:
                    print('No likely improvement')
                    return x

            alpha2 = alpha3 / 2.0
            g2 = self.g(x - alpha2 * z)

            h1 = (g2 - g1) / alpha2
            h2 = (g3 - g2) / (alpha3 - alpha2)
            h3 = (h2 - h1) / alpha3

            alpha0 = (alpha2 - h1 / h3) / 2.0
            g0 = self.g(x - alpha0 * z)

            alpha = alpha0
            g = g0
            if g3 < g:
                alpha = alpha3
                g = g3

            x = x - alpha * z
            residual = linalg.norm(x - prev_x, np.inf)
            row = len(df)
            df.loc[row] = [nxe for nxe in x] + [g, residual, np.nan]
            if math.fabs(g - g1) < TOR:
                break

        for i in range(len(df)):
            xk = np.array([df.loc[i][j] for j in range(len(x))])
            df.loc[i][4] = linalg.norm(xk - x, np.inf)

        print(self.f(x))
        return df

In [4]:
x0 = np.array([0, 0])
SteepestDescent().run(x0).astype(np.float64)

[-0.00020007  0.0009075 ]


,x1,x2,g,residual,actual-residual
0,0.00000000,0.00000000,128.00000000,nan,1.99977134
1,0.40926180,0.10231545,69.27847372,0.40926180,1.89745589
2,0.66283555,1.38995524,18.19552276,1.28763979,0.60981610
3,0.43320141,1.45993792,4.49808135,0.22963413,0.53983342
4,0.48412240,1.51291181,3.86258028,0.05297389,0.48685953
5,0.42827597,1.60719656,2.54955032,0.09428475,0.39257478
6,0.47108338,1.62644824,2.06973978,0.04280741,0.37332310
7,0.48109243,1.97435531,0.07833432,0.34790707,0.02541603
8,0.49695350,1.97377989,0.00917044,0.01586108,0.02599146
9,0.50008950,1.97979996,0.00706617,0.00602008,0.01997138


In [5]:
x0 = np.array([5, 5])
SteepestDescent().run(x0).astype(np.float64)

No likely improvement


array([ 4.50137356,  1.15513658])

## (b) 
$$\begin{align*}
\sin(4\pi x_1 x_2)-2x_2-x_1 &= 0 \\
\left( \frac{4\pi-1}{4\pi} \right)(\exp^{2x_1}-\exp)+4\exp x_2^2 -2\exp x_1 &= 0
\end{align*}$$

In [6]:
class SteepestDescent(SteepestDescentMethod):

    def __init__(self):
        super(SteepestDescentMethod, self).__init__()

    def f(self, x):
        sol = np.zeros(len(x))
        sol[0] = math.sin(4 * math.pi * x[0] * x[1]) - 2 * x[1] - x[0]
        sol[1] = ((4 * math.pi - 1) / (4 * math.pi)) * (math.exp(2 * x[0]) - math.e) + 4 * math.e * pow(x[1], 2) - 2 * math.e * x[0]
        return sol

    def jacobian(self, x):
        jac = np.zeros(shape=(2, 2))
        jac[0][0] = 4 * math.pi * x[1] * math.cos(4 * math.pi * x[0] * x[1]) - 1
        jac[0][1] = 4 * math.pi * x[0] * math.cos(4 * math.pi * x[0] * x[1]) - 2
        jac[1][0] = 2 * (4 * math.pi - 1) / (4 * math.pi) * math.exp(2 * x[0]) - 2 * math.e
        jac[1][1] = 8 * math.e * x[1]
        return jac

    def g(self, x):
        sol = self.f(x)
        return sum([e * e for e in sol])

    def grad_g(self, x):
        return 2 * self.jacobian(x).transpose().dot(self.f(x))

    def run(self, x):
        df = pd.DataFrame(columns=['x' + str(i + 1) for i in range(len(x))] + ['g', 'residual', 'actual-residual'])

        row = len(df)
        df.loc[row] = [xe for xe in x] + [self.g(x), np.nan, np.nan]

        while True:
            prev_x = x
            g1 = self.g(x)
            z = self.grad_g(x)
            z0 = linalg.norm(z, 2)
            if z0 == 0.0:
                print('Zero gradient')
                return x

            z /= z0
            alpha3 = 1
            g3 = self.g(x - alpha3 * z)
            while g3 >= g1:
                alpha3 /= 2.0
                g3 = self.g(x - alpha3 * z)
                if alpha3 < TOR / 2.0:
                    print('No likely improvement')
                    return x

            alpha2 = alpha3 / 2.0
            g2 = self.g(x - alpha2 * z)

            h1 = (g2 - g1) / alpha2
            h2 = (g3 - g2) / (alpha3 - alpha2)
            h3 = (h2 - h1) / alpha3

            alpha0 = (alpha2 - h1 / h3) / 2.0
            g0 = self.g(x - alpha0 * z)

            alpha = alpha0
            g = g0
            if g3 < g:
                alpha = alpha3
                g = g3

            x = x - alpha * z
            residual = linalg.norm(x - prev_x, np.inf)
            row = len(df)
            df.loc[row] = [nxe for nxe in x] + [g, residual, np.nan]
            if math.fabs(g - g1) < TOR:
                break

        for i in range(len(df)):
            xk = np.array([df.loc[i][j] for j in range(len(x))])
            df.loc[i][4] = linalg.norm(xk - x, np.inf)

        print(self.f(x))
        return df

In [7]:
x0 = np.array([0, 0])
SteepestDescent().run(x0).astype(np.float64)

[-0.00000966  0.00000187]


,x1,x2,g,residual,actual-residual
0,0.00000000,0.00000000,2.50128555,nan,0.37369823
1,-0.35792588,0.00000000,0.13938957,0.35792588,0.05626797
2,-0.36100921,0.05788368,0.00331642,0.05788368,0.01268902
3,-0.37370318,0.05683275,0.00001418,0.01269398,0.00056478
4,-0.37365538,0.05627157,0.00000004,0.00056118,0.00004285
5,-0.37369823,0.05626797,0.00000000,0.00004285,0.00000000


In [8]:
x0 = np.array([-10, -10])
SteepestDescent().run(x0).astype(np.float64)

[-0.00000295 -0.00001662]


,x1,x2,g,residual,actual-residual
0,-10.00000000,-10.00000000,"1,298,622.87059286",nan,10.05626676
1,-9.85030322,-6.23166204,"224,460.45002276",3.76833796,9.47660872
2,-9.73850971,-3.63703626,"38,002.93010292",2.59462578,9.36481520
3,-9.57475611,-1.79627799,"7,352.61856965",1.84075827,9.20106160
4,-9.42728129,-0.48084698,"2,744.48879915",1.31543101,9.05358678
5,-9.31427307,0.07822576,"2,402.52613683",0.55907274,8.94057856
6,-9.28263021,-0.04270285,"2,373.03991323",0.12092861,8.90893570
7,-8.96090564,0.34004158,"2,313.21884492",0.38274444,8.58721114
8,-8.95674868,0.34677641,"2,309.37009128",0.00673483,8.58305417
9,-7.40302032,-0.55298912,"1,775.72712083",1.55372835,7.02932582


In [9]:
x0 = np.array([1, -10])
SteepestDescent().run(x0).astype(np.float64)

[-0.00094747  0.0003531 ]


,x1,x2,g,residual,actual-residual
0,1.00000000,-10.00000000,"1,180,137.68151253",nan,9.50736624
1,0.89932050,-6.32337970,"187,552.64298358",3.67662030,5.83074593
2,0.77395070,-3.88312257,"26,152.76944100",2.44025712,3.39048881
3,0.75284256,-2.25227309,"2,788.10119944",1.63084948,1.75963933
4,0.66257732,-1.16190450,148.55882322,1.09036859,0.66927074
5,0.56349580,-0.41724529,0.69904636,0.74465921,0.15564059
6,0.56255023,-0.46020087,0.39844694,0.04295558,0.15469502
7,0.38500066,-0.45931872,0.16652615,0.17754957,0.03331504
8,0.38647846,-0.49257640,0.00763522,0.03325768,0.02137675
9,0.39463347,-0.49622444,0.00419297,0.00815501,0.01322174


In [10]:
x0 = np.array([-1, 1])
SteepestDescent().run(x0).astype(np.float64)

[-0.000012   -0.00009894]


,x1,x2,g,residual,actual-residual
0,-1.00000000,1.00000000,195.10867067,nan,0.94373274
1,-0.74472188,0.03313233,3.25901614,0.96686767,0.37104555
2,-0.12489440,0.22405344,0.75954690,0.61982748,0.24878193
3,-0.15513335,0.23488793,0.71990015,0.03023895,0.21854298
4,-0.24127452,0.00019728,0.44483196,0.23469065,0.13240182
5,-0.35561465,0.05707686,0.00672077,0.11434013,0.01806169
6,-0.37282006,0.05857054,0.00023516,0.01720541,0.00230327
7,-0.37304367,0.05628978,0.00000889,0.00228076,0.00063267
8,-0.37366873,0.05634828,0.00000030,0.00062507,0.00008102
9,-0.37367633,0.05626726,0.00000001,0.00008102,0.00000000


## (c) 
$$\begin{align*}
x_1(1-x_1)+4x_2 &= 12 \\
(x_1 - x_2)^2 + (2x_2-3)^2 &= 25
\end{align*}$$

In [11]:
class SteepestDescent(SteepestDescentMethod):

    def __init__(self):
        super(SteepestDescentMethod, self).__init__()

    def f(self, x):
        sol = np.zeros(len(x))
        sol[0] = x[0] * (1 - x[0]) + 4 * x[1] - 12
        sol[1] = pow(x[0] - x[1], 2) + pow(2 * x[1] - 3, 2) - 25
        return sol

    def jacobian(self, x):
        jac = np.zeros(shape=(2, 2))
        jac[0][0] = 1 - 2 * x[0]
        jac[0][1] = 4
        jac[1][0] = 2 * (x[0] - x[1])
        jac[1][1] = -2 * (x[0] * x[1]) + 4 * (2 * x[1] - 3)
        return jac

    def g(self, x):
        sol = self.f(x)
        return sum([e * e for e in sol])

    def grad_g(self, x):
        return 2 * self.jacobian(x).transpose().dot(self.f(x))

    def run(self, x):
        df = pd.DataFrame(columns=['x' + str(i + 1) for i in range(len(x))] + ['g', 'residual', 'actual-residual'])

        row = len(df)
        df.loc[row] = [xe for xe in x] + [self.g(x), np.nan, np.nan]

        while True:
            prev_x = x
            g1 = self.g(x)
            z = self.grad_g(x)
            z0 = linalg.norm(z, 2)
            if z0 == 0.0:
                print('Zero gradient')
                return x

            z /= z0
            alpha3 = 1
            g3 = self.g(x - alpha3 * z)
            while g3 >= g1:
                alpha3 /= 2.0
                g3 = self.g(x - alpha3 * z)
                if alpha3 < TOR / 2.0:
                    print('No likely improvement')
                    return x

            alpha2 = alpha3 / 2.0
            g2 = self.g(x - alpha2 * z)

            h1 = (g2 - g1) / alpha2
            h2 = (g3 - g2) / (alpha3 - alpha2)
            h3 = (h2 - h1) / alpha3

            alpha0 = (alpha2 - h1 / h3) / 2.0
            g0 = self.g(x - alpha0 * z)

            alpha = alpha0
            g = g0
            if g3 < g:
                alpha = alpha3
                g = g3

            x = x - alpha * z
            residual = linalg.norm(x - prev_x, np.inf)
            row = len(df)
            df.loc[row] = [nxe for nxe in x] + [g, residual, np.nan]
            if math.fabs(g - g1) < TOR:
                break

        for i in range(len(df)):
            xk = np.array([df.loc[i][j] for j in range(len(x))])
            df.loc[i][4] = linalg.norm(xk - x, np.inf)

        print(self.f(x))
        return df

In [12]:
x0 = np.array([0, 0])
SteepestDescent().run(x0).astype(np.float64)

No likely improvement


array([ 0.80017438, -0.71487857])

In [13]:
x0 = np.array([1, 5])
SteepestDescent().run(x0).astype(np.float64)

No likely improvement


array([ 1.59191552,  3.7219658 ])

In [14]:
x0 = np.array([-1, 500])
SteepestDescent().run(x0).astype(np.float64)

No likely improvement


array([  31.12282349,  340.09020578])

## (d) 
$$\begin{align*}
5x_1^2-x_2^2 &= 0 \\
x_2 - 0.25(\sin x_1 + \cos x_2) &= 0
\end{align*}$$

In [15]:
class SteepestDescent(SteepestDescentMethod):

    def __init__(self):
        super(SteepestDescentMethod, self).__init__()

    def f(self, x):
        sol = np.zeros(len(x))
        sol[0] = 5 * pow(x[0], 2) - pow(x[1], 2)
        sol[1] = x[1] - 0.25 * (math.sin(x[0]) + math.cos(x[1]))
        return sol

    def jacobian(self, x):
        jac = np.zeros(shape=(2, 2))
        jac[0][0] = 10 * x[0]
        jac[0][1] = -2 * x[1]
        jac[1][0] = -0.25 * math.cos(x[0])
        jac[1][1] = 1 + 0.25 * math.sin(x[1])
        return jac

    def g(self, x):
        sol = self.f(x)
        return sum([e * e for e in sol])

    def grad_g(self, x):
        return 2 * self.jacobian(x).transpose().dot(self.f(x))

    def run(self, x):
        df = pd.DataFrame(columns=['x' + str(i + 1) for i in range(len(x))] + ['g', 'residual', 'actual-residual'])

        row = len(df)
        df.loc[row] = [xe for xe in x] + [self.g(x), np.nan, np.nan]

        while True:
            prev_x = x
            g1 = self.g(x)
            z = self.grad_g(x)
            z0 = linalg.norm(z, 2)
            if z0 == 0.0:
                print('Zero gradient')
                return x

            z /= z0
            alpha3 = 1
            g3 = self.g(x - alpha3 * z)
            while g3 >= g1:
                alpha3 /= 2.0
                g3 = self.g(x - alpha3 * z)
                if alpha3 < TOR / 2.0:
                    print('No likely improvement')
                    return x

            alpha2 = alpha3 / 2.0
            g2 = self.g(x - alpha2 * z)

            h1 = (g2 - g1) / alpha2
            h2 = (g3 - g2) / (alpha3 - alpha2)
            h3 = (h2 - h1) / alpha3

            alpha0 = (alpha2 - h1 / h3) / 2.0
            g0 = self.g(x - alpha0 * z)

            alpha = alpha0
            g = g0
            if g3 < g:
                alpha = alpha3
                g = g3

            x = x - alpha * z
            residual = linalg.norm(x - prev_x, np.inf)
            row = len(df)
            df.loc[row] = [nxe for nxe in x] + [g, residual, np.nan]
            if math.fabs(g - g1) < TOR:
                break

        for i in range(len(df)):
            xk = np.array([df.loc[i][j] for j in range(len(x))])
            df.loc[i][4] = linalg.norm(xk - x, np.inf)

        print(self.f(x))
        return df

In [16]:
x0 = np.array([0, 0])
SteepestDescent().run(x0).astype(np.float64)

[-0.00007182  0.00002307]


,x1,x2,g,residual,actual-residual
0,0.00000000,0.00000000,0.06250000,nan,0.21953554
1,-0.05607755,0.22431019,0.00122590,0.22431019,0.04202855
2,-0.08643053,0.20997065,0.00021329,0.03035299,0.01167556
3,-0.09563509,0.22095960,0.00001039,0.01098894,0.00247101
4,-0.09776686,0.21914366,0.00000028,0.00213177,0.00039189
5,-0.09810610,0.21953554,0.00000001,0.00039189,0.00000000


In [17]:
x0 = np.array([1, 1])
SteepestDescent().run(x0).astype(np.float64)

[-0.00000376 -0.00001162]


,x1,x2,g,residual,actual-residual
0,1.00000000,1.00000000,16.42844444,nan,1.09815552
1,0.32594437,1.12172961,1.39940755,0.67405563,0.90223862
2,0.47028574,0.96715963,0.53594490,0.15456997,0.74766865
3,-0.30202986,0.33192060,0.14855017,0.77231559,0.20387434
4,0.19752416,0.35303426,0.00977782,0.49955401,0.29567968
5,0.15314729,0.34371460,0.00492878,0.04437687,0.25130281
6,0.16160171,0.30898780,0.00216867,0.03472680,0.25975723
7,-0.08211628,0.25329631,0.00193674,0.24371799,0.03380532
8,-0.09440997,0.21789093,0.00001539,0.03540538,0.00374555
9,-0.09810260,0.21952376,0.00000001,0.00369262,0.00005292


In [18]:
x0 = np.array([-1, -1])
SteepestDescent().run(x0).astype(np.float64)

[-0.00005302  0.00006253]


,x1,x2,g,residual,actual-residual
0,-1.00000000,-1.00000000,16.85508457,nan,1.27115921
1,-0.32655598,-1.12277883,1.85368655,0.67344402,1.39393804
2,-0.47326027,-0.98288612,1.03890416,0.14670429,1.25404533
3,0.23954383,-0.28152293,0.38067552,0.71280410,0.55268215
4,-0.17706816,-0.00505512,0.06909515,0.41661199,0.29829051
5,0.09920689,0.25108387,0.00044220,0.27627505,0.02201547
6,0.12065048,0.27272920,0.00000609,0.02164533,0.00156999
7,0.12165658,0.27152013,0.00000019,0.00120908,0.00043423
8,0.12122236,0.27115921,0.00000001,0.00043423,0.00000000


In [19]:
x0 = np.array([100, -100])
SteepestDescent().run(x0).astype(np.float64)

[ 0.00003071 -0.00001606]


,x1,x2,g,residual,actual-residual
0,100.00000000,-100.00000000,"1,600,010,017.80558038",nan,100.21947968
1,71.57253153,-105.68541049,"208,632,572.14402342",28.42746847,105.90489017
2,56.31336362,-110.19162532,"13,804,240.15237501",15.25916791,110.41110500
3,50.94467543,-112.29234463,"147,541.08674494",5.36868819,112.51182431
4,50.33699733,-112.55977060,"12,722.52334879",0.60767810,112.77925028
5,50.33806702,-112.55877959,"12,721.94341635",0.00106969,112.77825928
6,50.33514049,-112.55562070,"12,721.36308737",0.00315889,112.77510038
7,50.33621103,-112.55462893,"12,720.78227655",0.00107054,112.77410861
8,50.33328211,-112.55146737,"12,720.20106991",0.00316156,112.77094705
9,50.33435351,-112.55047483,"12,719.61938192",0.00107140,112.76995451
